In [1]:
import pandas as pd
import matplotlib.pyplot as plt

import numpy as np
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest

import warnings
warnings.filterwarnings("ignore")

In [3]:

data_dir = './'

train_df = pd.read_csv(data_dir+'iris_train.csv')
train_df.drop(['id'],axis=1,inplace=True)
test_df = pd.read_csv(data_dir+'iris_test.csv')

from imblearn.over_sampling import *
X=train_df.drop(['species'],axis=1)
y=train_df['species']

######################
iso = IsolationForest(contamination=0.01)
yhat = iso.fit_predict(X.values)
new_X = []
new_Y = []
for i,j,k in zip(X.values, yhat, y):
    if j==1:
        new_X.append(i)
        new_Y.append(k)
new_X = np.array(new_X)
new_X = pd.DataFrame(new_X)
new_X.columns = X.columns
X = new_X
new_Y = pd.Series(data=new_Y,name='species')
y = new_Y
########################


X_samp, y_samp = SMOTE(random_state=103).fit_resample(X, y)

train_df_ros = pd.DataFrame(X_samp, columns=X.columns)
train_df_ros['species'] = y_samp

train_df_ros = train_df_ros[train_df.columns]

train_df_ros_aug = train_df_ros.copy()

np.random.seed(seed=42)

train_df = train_df_ros_aug

In [4]:
encoder = LabelEncoder()
encoder.fit(train_df['species'])

train_df.species = encoder.transform(train_df.species)
test_df.species = encoder.transform(test_df.species)

In [5]:
df1=train_df[train_df['species']==0].drop(['species','sepal width (cm)','petal width (cm)'],axis=1)
df1 = pd.DataFrame(df1.values, columns=df1.columns)
df1['sepal width (cm)']=train_df[train_df['species']==0]['sepal width (cm)'].reset_index(drop=True)
df1['petal width (cm)']=train_df[train_df['species']==0]['petal width (cm)'].reset_index(drop=True)
df1['species']=0
df2=train_df[train_df['species']==1].drop(['species','sepal width (cm)','petal width (cm)'],axis=1)
df2 = pd.DataFrame(df2.values, columns=df2.columns)
df2['sepal width (cm)']=train_df[train_df['species']==1]['sepal width (cm)'].reset_index(drop=True)
df2['petal width (cm)']=train_df[train_df['species']==1]['petal width (cm)'].reset_index(drop=True)
df2['species']=1

df3=train_df[train_df['species']==2].drop(['species','sepal width (cm)','petal width (cm)'],axis=1)
df3 = pd.DataFrame(df3.values, columns=df3.columns)
df3['sepal width (cm)']=train_df[train_df['species']==2]['sepal width (cm)'].reset_index(drop=True)
df3['petal width (cm)']=train_df[train_df['species']==2]['petal width (cm)'].reset_index(drop=True)
df3['species']=2

df=pd.concat([df1,df2,df3])

df1=test_df[test_df['species']==0].drop(['id','species'],axis=1)
df1 = pd.DataFrame(df1.values, columns=df1.columns)
df1['id']=test_df[test_df['species']==0]['id'].reset_index(drop=True)
df1['species']=0

df2=test_df[test_df['species']==1].drop(['id','species'],axis=1)
df2 = pd.DataFrame(df2.values, columns=df2.columns)
df2['id']=test_df[test_df['species']==1]['id'].reset_index(drop=True)
df2['species']=1

df3=test_df[test_df['species']==2].drop(['id','species'],axis=1)
df3 = pd.DataFrame(df3.values, columns=df3.columns)
df3['id']=test_df[test_df['species']==2]['id'].reset_index(drop=True)
df3['species']=2

test_df=pd.concat([df1,df2,df3])
test_df=test_df.sort_values('id')
df=pd.get_dummies(df,columns=['species'])
test_df=pd.get_dummies(test_df,columns=['species'])

X=df.drop(['sepal width (cm)','petal width (cm)'],axis=1)
y1=df['sepal width (cm)']
y2=df['petal width (cm)']


In [6]:
import xgboost
#for y1
xgb_model1 = xgboost.XGBRegressor(base_score=0.5, booster='gblinear', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=0.75, missing=None, n_estimators=8000,
             n_jobs=1, nthread=None, objective='reg:squarederror',
             random_state=42, reg_alpha=0, reg_lambda=0, scale_pos_weight=1,
             seed=None, silent=None, subsample=0.5, verbosity=1)

#for y2
xgb_model2 = xgboost.XGBRegressor(base_score=0.5, booster='gblinear', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.2, max_delta_step=0,
             max_depth=3, min_child_weight=0.75, missing=None, n_estimators=8000,
             n_jobs=1, nthread=None, objective='reg:squarederror',
             random_state=42, reg_alpha=0, reg_lambda=0, scale_pos_weight=1,
             seed=None, silent=None, subsample=0.5, verbosity=1)

In [7]:
xgb_model2.fit(X,y2)
xgb_model1.fit(X,y1)

[18:16:32] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:576: 
Parameters: { "colsample_bylevel", "colsample_bynode", "colsample_bytree", "gamma", "max_delta_step", "max_depth", "min_child_weight", "subsample" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[18:16:32] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:576: 
Parameters: { "colsample_bylevel", "colsample_bynode", "colsample_bytree", "gamma", "max_delta_step", "max_depth", "min_child_weight", "subsample" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but get

XGBRegressor(base_score=0.5, booster='gblinear', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type='gain',
             interaction_constraints=None, learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=0.75, missing=None,
             monotone_constraints=None, n_estimators=8000, n_jobs=1, nthread=1,
             num_parallel_tree=None, predictor=None, random_state=42,
             reg_alpha=0, reg_lambda=0, scale_pos_weight=1, seed=42,
             silent=None, subsample=0.5, tree_method=None,
             validate_parameters=1, ...)

In [8]:
import catboost
cat_model1 = catboost.CatBoostRegressor(iterations=8000,learning_rate=0.1)
cat_model2 = catboost.CatBoostRegressor(iterations=8000,learning_rate=0.2)
#
cat_model2.fit(X,y2)
cat_model1.fit(X,y1)

0:	learn: 0.6575480	total: 55.1ms	remaining: 7m 20s
1:	learn: 0.5543553	total: 57.2ms	remaining: 3m 48s
2:	learn: 0.4780758	total: 57.6ms	remaining: 2m 33s
3:	learn: 0.4120882	total: 58.7ms	remaining: 1m 57s
4:	learn: 0.3508190	total: 59.2ms	remaining: 1m 34s
5:	learn: 0.3092578	total: 59.8ms	remaining: 1m 19s
6:	learn: 0.2699095	total: 60.1ms	remaining: 1m 8s
7:	learn: 0.2449304	total: 60.7ms	remaining: 1m
8:	learn: 0.2227999	total: 61.2ms	remaining: 54.4s
9:	learn: 0.2052699	total: 61.7ms	remaining: 49.3s
10:	learn: 0.1918758	total: 62.1ms	remaining: 45.1s
11:	learn: 0.1819296	total: 62.4ms	remaining: 41.5s
12:	learn: 0.1739518	total: 62.8ms	remaining: 38.6s
13:	learn: 0.1692571	total: 63.2ms	remaining: 36s
14:	learn: 0.1664730	total: 63.7ms	remaining: 33.9s
15:	learn: 0.1630238	total: 64.3ms	remaining: 32.1s
16:	learn: 0.1602265	total: 64.9ms	remaining: 30.5s
17:	learn: 0.1581688	total: 65.4ms	remaining: 29s
18:	learn: 0.1556921	total: 65.9ms	remaining: 27.7s
19:	learn: 0.1536454	to

247:	learn: 0.0542858	total: 190ms	remaining: 5.93s
248:	learn: 0.0542554	total: 190ms	remaining: 5.92s
249:	learn: 0.0542450	total: 191ms	remaining: 5.92s
250:	learn: 0.0542281	total: 191ms	remaining: 5.91s
251:	learn: 0.0541944	total: 192ms	remaining: 5.9s
252:	learn: 0.0541433	total: 192ms	remaining: 5.89s
253:	learn: 0.0540986	total: 193ms	remaining: 5.89s
254:	learn: 0.0540632	total: 194ms	remaining: 5.88s
255:	learn: 0.0540516	total: 194ms	remaining: 5.87s
256:	learn: 0.0540412	total: 195ms	remaining: 5.86s
257:	learn: 0.0540260	total: 195ms	remaining: 5.85s
258:	learn: 0.0540023	total: 196ms	remaining: 5.85s
259:	learn: 0.0539681	total: 196ms	remaining: 5.84s
260:	learn: 0.0539310	total: 197ms	remaining: 5.84s
261:	learn: 0.0539175	total: 198ms	remaining: 5.85s
262:	learn: 0.0539096	total: 199ms	remaining: 5.84s
263:	learn: 0.0538978	total: 199ms	remaining: 5.84s
264:	learn: 0.0538766	total: 200ms	remaining: 5.84s
265:	learn: 0.0538310	total: 201ms	remaining: 5.84s
266:	learn: 0

434:	learn: 0.0523337	total: 321ms	remaining: 5.59s
435:	learn: 0.0523315	total: 322ms	remaining: 5.58s
436:	learn: 0.0523303	total: 322ms	remaining: 5.58s
437:	learn: 0.0523271	total: 323ms	remaining: 5.58s
438:	learn: 0.0523267	total: 323ms	remaining: 5.57s
439:	learn: 0.0523251	total: 324ms	remaining: 5.57s
440:	learn: 0.0523247	total: 325ms	remaining: 5.57s
441:	learn: 0.0523243	total: 325ms	remaining: 5.56s
442:	learn: 0.0523213	total: 326ms	remaining: 5.56s
443:	learn: 0.0523209	total: 326ms	remaining: 5.55s
444:	learn: 0.0523190	total: 327ms	remaining: 5.55s
445:	learn: 0.0523187	total: 327ms	remaining: 5.54s
446:	learn: 0.0523172	total: 328ms	remaining: 5.54s
447:	learn: 0.0523154	total: 328ms	remaining: 5.54s
448:	learn: 0.0523144	total: 329ms	remaining: 5.53s
449:	learn: 0.0523114	total: 329ms	remaining: 5.52s
450:	learn: 0.0523106	total: 330ms	remaining: 5.52s
451:	learn: 0.0523079	total: 330ms	remaining: 5.51s
452:	learn: 0.0523072	total: 331ms	remaining: 5.51s
453:	learn: 

687:	learn: 0.0521826	total: 452ms	remaining: 4.8s
688:	learn: 0.0521825	total: 453ms	remaining: 4.8s
689:	learn: 0.0521823	total: 453ms	remaining: 4.8s
690:	learn: 0.0521822	total: 454ms	remaining: 4.8s
691:	learn: 0.0521821	total: 455ms	remaining: 4.8s
692:	learn: 0.0521821	total: 455ms	remaining: 4.8s
693:	learn: 0.0521819	total: 456ms	remaining: 4.8s
694:	learn: 0.0521819	total: 456ms	remaining: 4.8s
695:	learn: 0.0521817	total: 457ms	remaining: 4.79s
696:	learn: 0.0521816	total: 457ms	remaining: 4.79s
697:	learn: 0.0521815	total: 458ms	remaining: 4.79s
698:	learn: 0.0521815	total: 458ms	remaining: 4.79s
699:	learn: 0.0521815	total: 459ms	remaining: 4.78s
700:	learn: 0.0521814	total: 459ms	remaining: 4.78s
701:	learn: 0.0521813	total: 460ms	remaining: 4.78s
702:	learn: 0.0521812	total: 460ms	remaining: 4.78s
703:	learn: 0.0521812	total: 461ms	remaining: 4.77s
704:	learn: 0.0521812	total: 461ms	remaining: 4.77s
705:	learn: 0.0521811	total: 462ms	remaining: 4.77s
706:	learn: 0.052181

931:	learn: 0.0521753	total: 579ms	remaining: 4.39s
932:	learn: 0.0521753	total: 579ms	remaining: 4.39s
933:	learn: 0.0521753	total: 580ms	remaining: 4.39s
934:	learn: 0.0521753	total: 581ms	remaining: 4.39s
935:	learn: 0.0521753	total: 581ms	remaining: 4.39s
936:	learn: 0.0521753	total: 582ms	remaining: 4.38s
937:	learn: 0.0521753	total: 582ms	remaining: 4.38s
938:	learn: 0.0521753	total: 583ms	remaining: 4.38s
939:	learn: 0.0521753	total: 583ms	remaining: 4.38s
940:	learn: 0.0521753	total: 584ms	remaining: 4.38s
941:	learn: 0.0521753	total: 584ms	remaining: 4.38s
942:	learn: 0.0521753	total: 585ms	remaining: 4.38s
943:	learn: 0.0521752	total: 585ms	remaining: 4.37s
944:	learn: 0.0521752	total: 586ms	remaining: 4.37s
945:	learn: 0.0521752	total: 586ms	remaining: 4.37s
946:	learn: 0.0521752	total: 587ms	remaining: 4.37s
947:	learn: 0.0521752	total: 587ms	remaining: 4.37s
948:	learn: 0.0521752	total: 588ms	remaining: 4.37s
949:	learn: 0.0521752	total: 588ms	remaining: 4.36s
950:	learn: 

...

6868:	learn: 0.0839974	total: 4.5s	remaining: 742ms
6869:	learn: 0.0839974	total: 4.5s	remaining: 741ms
6870:	learn: 0.0839974	total: 4.51s	remaining: 740ms
6871:	learn: 0.0839974	total: 4.51s	remaining: 740ms
6872:	learn: 0.0839974	total: 4.51s	remaining: 739ms
6873:	learn: 0.0839974	total: 4.51s	remaining: 738ms
6874:	learn: 0.0839974	total: 4.51s	remaining: 738ms
6875:	learn: 0.0839974	total: 4.51s	remaining: 737ms
6876:	learn: 0.0839974	total: 4.51s	remaining: 736ms
6877:	learn: 0.0839974	total: 4.51s	remaining: 736ms
6878:	learn: 0.0839974	total: 4.51s	remaining: 735ms
6879:	learn: 0.0839974	total: 4.51s	remaining: 734ms
6880:	learn: 0.0839974	total: 4.51s	remaining: 734ms
6881:	learn: 0.0839974	total: 4.51s	remaining: 733ms
6882:	learn: 0.0839974	total: 4.51s	remaining: 732ms
6883:	learn: 0.0839974	total: 4.51s	remaining: 732ms
6884:	learn: 0.0839974	total: 4.51s	remaining: 731ms
6885:	learn: 0.0839974	total: 4.51s	remaining: 730ms
6886:	learn: 0.0839974	total: 4.51s	remaining: 7

7096:	learn: 0.0839974	total: 4.64s	remaining: 590ms
7097:	learn: 0.0839974	total: 4.64s	remaining: 589ms
7098:	learn: 0.0839974	total: 4.64s	remaining: 589ms
7099:	learn: 0.0839974	total: 4.64s	remaining: 588ms
7100:	learn: 0.0839974	total: 4.64s	remaining: 587ms
7101:	learn: 0.0839974	total: 4.64s	remaining: 587ms
7102:	learn: 0.0839974	total: 4.64s	remaining: 586ms
7103:	learn: 0.0839974	total: 4.64s	remaining: 585ms
7104:	learn: 0.0839974	total: 4.64s	remaining: 585ms
7105:	learn: 0.0839974	total: 4.64s	remaining: 584ms
7106:	learn: 0.0839974	total: 4.64s	remaining: 583ms
7107:	learn: 0.0839974	total: 4.64s	remaining: 583ms
7108:	learn: 0.0839974	total: 4.64s	remaining: 582ms
7109:	learn: 0.0839974	total: 4.64s	remaining: 581ms
7110:	learn: 0.0839974	total: 4.64s	remaining: 581ms
7111:	learn: 0.0839974	total: 4.64s	remaining: 580ms
7112:	learn: 0.0839974	total: 4.65s	remaining: 579ms
7113:	learn: 0.0839974	total: 4.65s	remaining: 579ms
7114:	learn: 0.0839974	total: 4.65s	remaining:

7320:	learn: 0.0839974	total: 4.77s	remaining: 443ms
7321:	learn: 0.0839974	total: 4.77s	remaining: 442ms
7322:	learn: 0.0839974	total: 4.78s	remaining: 441ms
7323:	learn: 0.0839974	total: 4.78s	remaining: 441ms
7324:	learn: 0.0839974	total: 4.78s	remaining: 441ms
7325:	learn: 0.0839974	total: 4.78s	remaining: 440ms
7326:	learn: 0.0839974	total: 4.78s	remaining: 439ms
7327:	learn: 0.0839974	total: 4.78s	remaining: 439ms
7328:	learn: 0.0839974	total: 4.78s	remaining: 438ms
7329:	learn: 0.0839974	total: 4.78s	remaining: 437ms
7330:	learn: 0.0839974	total: 4.78s	remaining: 437ms
7331:	learn: 0.0839974	total: 4.79s	remaining: 436ms
7332:	learn: 0.0839974	total: 4.79s	remaining: 435ms
7333:	learn: 0.0839974	total: 4.79s	remaining: 435ms
7334:	learn: 0.0839974	total: 4.79s	remaining: 434ms
7335:	learn: 0.0839974	total: 4.79s	remaining: 433ms
7336:	learn: 0.0839974	total: 4.79s	remaining: 433ms
7337:	learn: 0.0839974	total: 4.79s	remaining: 432ms
7338:	learn: 0.0839974	total: 4.79s	remaining:

7552:	learn: 0.0839974	total: 4.91s	remaining: 291ms
7553:	learn: 0.0839974	total: 4.91s	remaining: 290ms
7554:	learn: 0.0839974	total: 4.91s	remaining: 289ms
7555:	learn: 0.0839974	total: 4.91s	remaining: 289ms
7556:	learn: 0.0839974	total: 4.91s	remaining: 288ms
7557:	learn: 0.0839974	total: 4.91s	remaining: 287ms
7558:	learn: 0.0839974	total: 4.91s	remaining: 287ms
7559:	learn: 0.0839974	total: 4.91s	remaining: 286ms
7560:	learn: 0.0839974	total: 4.92s	remaining: 285ms
7561:	learn: 0.0839974	total: 4.92s	remaining: 285ms
7562:	learn: 0.0839974	total: 4.92s	remaining: 284ms
7563:	learn: 0.0839974	total: 4.92s	remaining: 283ms
7564:	learn: 0.0839974	total: 4.92s	remaining: 283ms
7565:	learn: 0.0839974	total: 4.92s	remaining: 282ms
7566:	learn: 0.0839974	total: 4.92s	remaining: 282ms
7567:	learn: 0.0839974	total: 4.92s	remaining: 281ms
7568:	learn: 0.0839974	total: 4.92s	remaining: 280ms
7569:	learn: 0.0839974	total: 4.92s	remaining: 280ms
7570:	learn: 0.0839974	total: 4.92s	remaining:

7780:	learn: 0.0839974	total: 5.04s	remaining: 142ms
7781:	learn: 0.0839974	total: 5.04s	remaining: 141ms
7782:	learn: 0.0839974	total: 5.05s	remaining: 141ms
7783:	learn: 0.0839974	total: 5.05s	remaining: 140ms
7784:	learn: 0.0839974	total: 5.05s	remaining: 139ms
7785:	learn: 0.0839974	total: 5.05s	remaining: 139ms
7786:	learn: 0.0839974	total: 5.05s	remaining: 138ms
7787:	learn: 0.0839974	total: 5.05s	remaining: 137ms
7788:	learn: 0.0839974	total: 5.05s	remaining: 137ms
7789:	learn: 0.0839974	total: 5.05s	remaining: 136ms
7790:	learn: 0.0839974	total: 5.05s	remaining: 135ms
7791:	learn: 0.0839974	total: 5.05s	remaining: 135ms
7792:	learn: 0.0839974	total: 5.05s	remaining: 134ms
7793:	learn: 0.0839974	total: 5.05s	remaining: 134ms
7794:	learn: 0.0839974	total: 5.05s	remaining: 133ms
7795:	learn: 0.0839974	total: 5.05s	remaining: 132ms
7796:	learn: 0.0839974	total: 5.05s	remaining: 132ms
7797:	learn: 0.0839974	total: 5.05s	remaining: 131ms
7798:	learn: 0.0839974	total: 5.05s	remaining:

In [9]:
import ngboost


ng1 = ngboost.NGBRegressor(learning_rate=0.01,minibatch_frac=0.1,n_estimators=1000,random_state=42)
ng2 = ngboost.NGBRegressor(learning_rate=0.02,random_state=42)
ng1.fit(X,y1)
ng2.fit(X,y2)

[iter 0] loss=0.4620 val_loss=0.0000 scale=2.0000 norm=1.0650
[iter 100] loss=0.1832 val_loss=0.0000 scale=2.0000 norm=1.3822
[iter 200] loss=-0.3859 val_loss=0.0000 scale=2.0000 norm=0.6878
[iter 300] loss=-0.3259 val_loss=0.0000 scale=2.0000 norm=1.0708
[iter 400] loss=-0.2500 val_loss=0.0000 scale=1.0000 norm=0.4651
[iter 500] loss=-0.3406 val_loss=0.0000 scale=2.0000 norm=1.0666
[iter 600] loss=-0.4088 val_loss=0.0000 scale=1.0000 norm=0.4755
[iter 700] loss=-1.0568 val_loss=0.0000 scale=1.0000 norm=0.3906
[iter 800] loss=-0.5176 val_loss=0.0000 scale=2.0000 norm=0.8474
[iter 900] loss=-0.5896 val_loss=0.0000 scale=2.0000 norm=0.6313
[iter 0] loss=1.1546 val_loss=0.0000 scale=2.0000 norm=1.6270
[iter 100] loss=-0.6514 val_loss=0.0000 scale=2.0000 norm=0.8057
[iter 200] loss=-1.4869 val_loss=0.0000 scale=2.0000 norm=0.7782
[iter 300] loss=-1.8926 val_loss=0.0000 scale=2.0000 norm=0.7492
[iter 400] loss=-2.0931 val_loss=0.0000 scale=1.0000 norm=0.3625


NGBRegressor(learning_rate=0.02,
             random_state=RandomState(MT19937) at 0x1E081FA1940)

ann1 모델과 ann2 모델을 학습시킵니다. save폴더을 만들고
체크포인트로 파일을 저장한 뒤 제일 마지막에서 두번째 체크포인트를 사용합니다.
나머지는 삭제

In [ ]:
import torch as tor
from adabelief_pytorch import AdaBelief
from torch.utils.data import Dataset,DataLoader


class ANN1(tor.nn.Module):
    def __init__(self,):
        super().__init__()
        self.linear1 = tor.nn.Linear(5,12)
        self.linear2 = tor.nn.Linear(12,12)
        self.linear3 = tor.nn.Linear(12,12)
        self.linear4 = tor.nn.Linear(12, 12)
        self.linear5 = tor.nn.Linear(12, 12)
        self.linear6 = tor.nn.Linear(12,2)
        self.relu = tor.nn.ReLU()
        self.silu = tor.nn.SiLU()
        self.gelu = tor.nn.GELU()
        self.mi = tor.nn.Mish()
        self.norm = tor.nn.BatchNorm1d(12)


    def forward(self,x):

        out = self.silu(self.linear1(x))

        # out = self.norm(out)


        x = self.silu(self.linear2(out))

        # x = x+out

        x = self.silu(self.linear3(x))

        x = self.silu(self.linear4(x))

        x = self.silu(self.linear5(x))

        x = self.linear6(x)

        return x


class ANN2(tor.nn.Module):
    def __init__(self,):
        super().__init__()
        self.linear1 = tor.nn.Linear(5,16)
        self.linear2 = tor.nn.Linear(16,16)
        self.linear3 = tor.nn.Linear(16,16)
        self.linear4 = tor.nn.Linear(16, 16)
        self.linear5 = tor.nn.Linear(16,2)
        self.relu = tor.nn.ReLU()
        self.silu = tor.nn.SiLU()
        self.gelu = tor.nn.GELU()
        self.mi = tor.nn.Mish()
        self.norm = tor.nn.BatchNorm1d(12)


    def forward(self,x):

        out = self.relu(self.linear1(x))

        x = self.relu(self.linear2(out))

        x = self.relu(self.linear3(x))

        x = self.relu(self.linear4(x))

        x = self.linear5(x)


        return x

class SET(Dataset):
    def __init__(self,x,y):
        self.x = x
        self.y = y

    def __len__(self):
        return len(self.x)

    def __getitem__(self,idx):
        out ={}
        i = self.x[idx]
        o = self.y[idx]
        out['input'] = i
        out['target'] = o
        return out

import random
import os

def seed_everything(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    tor.manual_seed(seed)
    tor.cuda.manual_seed(seed)  # type: ignore
    tor.backends.cudnn.deterministic = True  # type: ignore
    tor.backends.cudnn.benchmark = True  # type: ignore

kf = KFold(n_splits=len(X), shuffle=False)
seed_everything(389)
cri = tor.nn.L1Loss()
yr = pd.concat([y1,y2],axis=1)
train_start = False
if train_start:
    for idx, (ti, vi) in enumerate(kf.split(X)):
        if idx == 0:
            xt, yt = np.array(X)[ti], np.array(yr)[ti]
            xv, yv = np.array(X)[vi], np.array(yr)[vi]
            n1,n2,n3 = xv[:,0][0],xv[:,1][0],xv[:,2][0]

            ts = SET(xt, yt)
            vs = SET(xv, yv)
            tl = DataLoader(ts,batch_size=len(xt),shuffle=True)
            vl = DataLoader(vs,batch_size=len(xv),shuffle=False)
            pt = 100.
            pat = 0
            # ann = ANN1()
            ann = ANN2()
            opt = AdaBelief(ann.parameters(), lr=1e-2, print_change_log=False, weight_decay=1e-3, weight_decouple=True)
            for epoch in range(1000):
                st, vt = 0., 0.
                for batch, gen in enumerate(tl):
                    opt.zero_grad()
                    ann.train()
                    pred = ann(gen['input'].float())
                    loss = cri(pred,gen['target'].float())
                    loss.backward()
                    opt.step()
                    st += loss.item()/(batch+1)


                for batch, gen in enumerate(vl):
                    ann.eval()
                    with tor.no_grad():
                        pred = ann(gen['input'].float())
                        loss = cri(pred,gen['target'].float())
                    vt += loss.item()/(batch+1)


                print("Epoch:",epoch,"loss:",st,"val_loss:",vt)

                if float(vt) < pt:
                    pt = float(vt)
                    pat = 0
                    tor.save(ann.state_dict(), f'./save/{n1}-{n2}-{n3}-{epoch}-ann1.pt') #1번째 파일생성
#                     tor.save(ann.state_dict(), f'./save/{n1}-{n2}-{n3}-{epoch}-ann2.pt') #2번째 파일생성

                else:
                    pat +=1

                if (pat>300) and (epoch>2500):
                    break


첫번째 방법은 catboost 절반과 ann1과 xgboost반반의 절반을 중합시킵니다.
두번째 방법은 catboost xgboost ngboost ann1 ann2 평균을 사용합니다.

In [ ]:
ann1 = ANN1()
test_check = tor.load('./save/4.4-1.4-1.0-866-ann1.pt')
ann1.load_state_dict(test_check)
pred1 = ann1(tor.from_numpy(np.array(test_df)).float())
pred1 = pred1.detach().cpu().numpy()
ann2 = ANN2()
test_check = tor.load('./save/4.4-1.4-1.0-924-ann2.pt')
ann2.load_state_dict(test_check)
pred2 = ann2(tor.from_numpy(np.array(test_df)).float())
pred2 = pred2.detach().cpu().numpy()

tmp_pred1 = xgb_model2.predict(test_df)
tmp_pred2 = xgb_model1.predict(test_df)
tmp_pred3 = cat_model2.predict(test_df)
tmp_pred4 = cat_model1.predict(test_df)
tmp_pred5 = ng2.predict(test_df)
tmp_pred6 = ng1.predict(test_df)
tmp_pred = np.concatenate([tmp_pred2.reshape(-1,1),tmp_pred1.reshape(-1,1)],axis=1)
tmp_pred_v2 =  np.concatenate([tmp_pred4.reshape(-1,1),tmp_pred3.reshape(-1,1)],axis=1)
tmp_pred_v3 =  np.concatenate([tmp_pred6.reshape(-1,1),tmp_pred5.reshape(-1,1)],axis=1)

pred = ((pred1+tmp_pred) / 2) / 2 + tmp_pred_v2 / 2 #첫번째 방법
pred = (pred1 + pred2 + tmp_pred + tmp_pred_v2 + tmp_pred_v3) / 5 #두번째 방법

sub = pd.read_csv('./sample_submission.csv')
sub.loc[:,'sepal width (cm)'] = pred[:,0]
sub.loc[:,'petal width (cm)'] = pred[:,1]
print(sub)
sub.to_csv('./submitlast.csv',index=False)